In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-04-13 03:08:38,550 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield text
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield text
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield text
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield text
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield text
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield text
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield text
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield text
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [5]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-04-13 03:09:00,435 : INFO : loading Dictionary object from stem100_dictionary
2018-04-13 03:09:00,604 : INFO : loaded stem100_dictionary


In [6]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-04-13 03:09:00,745 : INFO : loaded corpus index from stem100.mm.index
2018-04-13 03:09:00,746 : INFO : initializing corpus reader from stem100.mm
2018-04-13 03:09:00,771 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [7]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [8]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [9]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [10]:
%%time
model = gensim.models.Word2Vec(iter_token_corpus(), sg = 1, size=300, iter = 20, min_count=0, sample=.00001, workers=8)

2018-04-13 03:09:10,744 : INFO : collecting all words and their counts
2018-04-13 03:09:10,858 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-13 03:09:32,637 : INFO : PROGRESS: at sentence #10000, processed 11872635 words, keeping 28140 word types
2018-04-13 03:09:44,236 : INFO : PROGRESS: at sentence #20000, processed 22389436 words, keeping 31281 word types
2018-04-13 03:09:54,987 : INFO : PROGRESS: at sentence #30000, processed 32260628 words, keeping 32838 word types
2018-04-13 03:10:03,656 : INFO : PROGRESS: at sentence #40000, processed 40518216 words, keeping 34451 word types
2018-04-13 03:10:12,516 : INFO : PROGRESS: at sentence #50000, processed 47137590 words, keeping 35170 word types
2018-04-13 03:10:19,699 : INFO : PROGRESS: at sentence #60000, processed 53235450 words, keeping 36026 word types
2018-04-13 03:10:25,459 : INFO : PROGRESS: at sentence #70000, processed 58570547 words, keeping 36380 word types
2018-04-13 03:10:32,480 : INFO :

2018-04-13 03:14:35,407 : INFO : PROGRESS: at 0.27% examples, 170086 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:36,412 : INFO : PROGRESS: at 0.28% examples, 169785 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:37,413 : INFO : PROGRESS: at 0.29% examples, 169461 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:38,413 : INFO : PROGRESS: at 0.30% examples, 169097 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:39,429 : INFO : PROGRESS: at 0.31% examples, 168390 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:40,433 : INFO : PROGRESS: at 0.32% examples, 168160 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:41,439 : INFO : PROGRESS: at 0.33% examples, 167706 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:42,442 : INFO : PROGRESS: at 0.33% examples, 167210 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:43,442 : INFO : PROGRESS: at 0.34% examples, 166800 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:14:44,452 : INFO : PROGRESS: at 0.35% examples, 166847 words/s, in_q

2018-04-13 03:15:56,998 : INFO : PROGRESS: at 1.31% examples, 178212 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:15:58,004 : INFO : PROGRESS: at 1.32% examples, 178183 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:15:59,017 : INFO : PROGRESS: at 1.33% examples, 178347 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:00,024 : INFO : PROGRESS: at 1.34% examples, 178448 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:01,025 : INFO : PROGRESS: at 1.35% examples, 178627 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:02,027 : INFO : PROGRESS: at 1.36% examples, 178752 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:03,031 : INFO : PROGRESS: at 1.37% examples, 178759 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:04,039 : INFO : PROGRESS: at 1.38% examples, 178714 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:05,045 : INFO : PROGRESS: at 1.39% examples, 178769 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:16:06,052 : INFO : PROGRESS: at 1.40% examples, 178753 words/s, in_q

2018-04-13 03:17:18,721 : INFO : PROGRESS: at 2.45% examples, 183567 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:19,728 : INFO : PROGRESS: at 2.47% examples, 183537 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:20,739 : INFO : PROGRESS: at 2.48% examples, 183547 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:21,750 : INFO : PROGRESS: at 2.48% examples, 183540 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:22,768 : INFO : PROGRESS: at 2.49% examples, 183488 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:23,771 : INFO : PROGRESS: at 2.51% examples, 183497 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:24,780 : INFO : PROGRESS: at 2.53% examples, 183536 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:25,786 : INFO : PROGRESS: at 2.55% examples, 183565 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:26,787 : INFO : PROGRESS: at 2.56% examples, 183625 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:17:27,802 : INFO : PROGRESS: at 2.58% examples, 183643 words/s, in_q

2018-04-13 03:18:40,382 : INFO : PROGRESS: at 3.21% examples, 181260 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:41,385 : INFO : PROGRESS: at 3.30% examples, 181266 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:42,394 : INFO : PROGRESS: at 3.40% examples, 181319 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:43,405 : INFO : PROGRESS: at 3.47% examples, 181309 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:44,419 : INFO : PROGRESS: at 3.53% examples, 181284 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:45,427 : INFO : PROGRESS: at 3.57% examples, 181246 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:46,445 : INFO : PROGRESS: at 3.64% examples, 181245 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:47,446 : INFO : PROGRESS: at 3.73% examples, 181287 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:48,447 : INFO : PROGRESS: at 3.82% examples, 181292 words/s, in_qsize 0, out_qsize 0
2018-04-13 03:18:49,450 : INFO : PROGRESS: at 3.87% examples, 181266 words/s, in_q

...

2018-04-13 05:29:17,662 : INFO : PROGRESS: at 96.41% examples, 170468 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:18,671 : INFO : PROGRESS: at 96.42% examples, 170468 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:19,679 : INFO : PROGRESS: at 96.43% examples, 170470 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:20,688 : INFO : PROGRESS: at 96.44% examples, 170473 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:21,695 : INFO : PROGRESS: at 96.45% examples, 170474 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:22,704 : INFO : PROGRESS: at 96.46% examples, 170477 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:23,714 : INFO : PROGRESS: at 96.47% examples, 170479 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:24,726 : INFO : PROGRESS: at 96.48% examples, 170482 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:25,727 : INFO : PROGRESS: at 96.49% examples, 170486 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:29:26,731 : INFO : PROGRESS: at 96.50% examples, 170488 wor

2018-04-13 05:30:38,369 : INFO : PROGRESS: at 97.60% examples, 170711 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:39,373 : INFO : PROGRESS: at 97.61% examples, 170711 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:40,381 : INFO : PROGRESS: at 97.62% examples, 170713 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:41,381 : INFO : PROGRESS: at 97.63% examples, 170713 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:42,390 : INFO : PROGRESS: at 97.63% examples, 170713 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:43,401 : INFO : PROGRESS: at 97.64% examples, 170714 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:44,407 : INFO : PROGRESS: at 97.65% examples, 170715 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:45,408 : INFO : PROGRESS: at 97.66% examples, 170714 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:46,416 : INFO : PROGRESS: at 97.66% examples, 170716 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:30:47,419 : INFO : PROGRESS: at 97.67% examples, 170716 wor

2018-04-13 05:31:59,016 : INFO : PROGRESS: at 99.00% examples, 170800 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:00,029 : INFO : PROGRESS: at 99.09% examples, 170802 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:01,040 : INFO : PROGRESS: at 99.12% examples, 170805 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:02,051 : INFO : PROGRESS: at 99.14% examples, 170809 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:03,062 : INFO : PROGRESS: at 99.16% examples, 170814 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:04,074 : INFO : PROGRESS: at 99.19% examples, 170818 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:05,075 : INFO : PROGRESS: at 99.21% examples, 170823 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:06,076 : INFO : PROGRESS: at 99.24% examples, 170827 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:07,080 : INFO : PROGRESS: at 99.26% examples, 170833 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:32:08,094 : INFO : PROGRESS: at 99.29% examples, 170838 wor

2018-04-13 05:33:19,826 : INFO : PROGRESS: at 99.84% examples, 170955 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:20,831 : INFO : PROGRESS: at 99.85% examples, 170957 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:21,839 : INFO : PROGRESS: at 99.85% examples, 170958 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:22,843 : INFO : PROGRESS: at 99.86% examples, 170958 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:23,844 : INFO : PROGRESS: at 99.86% examples, 170959 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:24,859 : INFO : PROGRESS: at 99.87% examples, 170960 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:25,869 : INFO : PROGRESS: at 99.87% examples, 170959 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:26,872 : INFO : PROGRESS: at 99.88% examples, 170960 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:27,879 : INFO : PROGRESS: at 99.89% examples, 170961 words/s, in_qsize 0, out_qsize 0
2018-04-13 05:33:28,880 : INFO : PROGRESS: at 99.89% examples, 170962 wor

CPU times: user 6h 39min 33s, sys: 2min 27s, total: 6h 42min
Wall time: 2h 24min 34s


In [11]:
model.save('SGword2vecsubsam')

2018-04-13 05:33:45,009 : INFO : saving Word2Vec object under SGword2vecsubsam, separately None
2018-04-13 05:33:45,010 : INFO : not storing attribute syn0norm
2018-04-13 05:33:45,010 : INFO : storing np array 'syn0' to SGword2vecsubsam.wv.syn0.npy
2018-04-13 05:33:45,148 : INFO : storing np array 'syn1neg' to SGword2vecsubsam.syn1neg.npy
2018-04-13 05:33:45,203 : INFO : not storing attribute cum_table
2018-04-13 05:33:45,603 : INFO : saved SGword2vecsubsam


In [12]:
model = gensim.models.Word2Vec.load('SGword2vecsubsam')

2018-04-13 05:33:45,607 : INFO : loading Word2Vec object from SGword2vecsubsam
2018-04-13 05:33:45,688 : INFO : loading wv recursively from SGword2vecsubsam.wv.* with mmap=None
2018-04-13 05:33:45,689 : INFO : loading syn0 from SGword2vecsubsam.wv.syn0.npy with mmap=None
2018-04-13 05:33:45,705 : INFO : setting ignored attribute syn0norm to None
2018-04-13 05:33:45,706 : INFO : loading syn1neg from SGword2vecsubsam.syn1neg.npy with mmap=None
2018-04-13 05:33:45,720 : INFO : setting ignored attribute cum_table to None
2018-04-13 05:33:45,721 : INFO : loaded SGword2vecsubsam


In [105]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

IndentationError: unexpected indent (<ipython-input-105-b0cfacef251c>, line 2)

In [13]:
#preprocess a word
def T(word):
    if word not in en_stop:
        return ps.stem(word)

In [14]:
%%time

import math
from six import iteritems
from six.moves import xrange
idf = {}
for word, freq in dictionary.dfs.iteritems():
    idf[dictionary.id2token[word]] = math.log(dictionary.num_docs) - math.log(freq)


CPU times: user 32 ms, sys: 20 ms, total: 52 ms
Wall time: 34.4 ms


In [15]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [16]:
w2v = model.wv
def vec(document):
    return np.add.reduce([idf[i]*w2v[i] for i in document if i in dictionary.token2id])

In [17]:
import sys
sys.getsizeof(doc_vec)

NameError: name 'doc_vec' is not defined

In [18]:
%%time
doc_vec = []
for x in iter_token_corpus():
    x = vec(x)
    doc_vec.append(x)

CPU times: user 18min 58s, sys: 4.08 s, total: 19min 2s
Wall time: 19min 1s


In [19]:
from scipy.spatial.distance import cosine
def get_scores(query):
    scores = []
    i = 0
    query = vec(query)
    for x in doc_vec:
        if isinstance(x, np.ndarray):
            score = 1 - cosine(x, query)
        else:
            score = -1000
        scores.append(score)
    return scores

In [20]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        

CPU times: user 1min 21s, sys: 40 ms, total: 1min 21s
Wall time: 1min 22s


In [21]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.78286533363668942, 1979],
  ['ConsumerCourt_DCDRC_106530', '0', 0.72209370719154109, 9355],
  ['ConsumerCourt_DCDRC_107608', '1', 0.7961394711767289, 1244],
  ['ConsumerCourt_DCDRC_114291', '1', 0.83096922449134814, 219],
  ['ConsumerCourt_DCDRC_114382', '1', 0.80720326119544694, 818],
  ['ConsumerCourt_DCDRC_118185', '1', 0.81890328368091903, 484],
  ['ConsumerCourt_DCDRC_130318', '0', 0.81851252875889047, 495],
  ['ConsumerCourt_DCDRC_130570', '1', 0.80621130603300462, 853],
  ['ConsumerCourt_DCDRC_131146', '1', 0.85067170740041731, 44],
  ['ConsumerCourt_DCDRC_131717', '1', 0.84805313518393188, 62],
  ['ConsumerCourt_DCDRC_131741', '1', 0.81664429035969988, 534],
  ['ConsumerCourt_DCDRC_131818', '1', 0.8336749408524371, 184],
  ['ConsumerCourt_DCDRC_131950', '0', 0.81580699134997348, 550],
  ['ConsumerCourt_DCDRC_131972', '0', 0.85551673360999969, 24],
  ['ConsumerCourt_DCDRC_132932', '0', 0.87519394603036782, 1],
  ['ConsumerCourt_DCDRC_

In [22]:
model_name = 'SGw2v_idf_subsam'
# Store results in 1st sheet
wb = Workbook()
dest_filename = model_name + '.xlsx'
ws1 = wb.active
ws1.title = " "
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)


#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [23]:
x = f.parse(1)

In [24]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
f2 = pd.ExcelFile('all_results.xlsx')
append_df_to_excel("all_results.xlsx", f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel("all_results.xlsx", f.parse(2), sheet_name= f2.sheet_names[1])